In [1]:
# !pip install owlrl
import os

# Specify the desired directory
directory_path = r"C:\backup\narrateastory"

# Change the current working directory
os.chdir(directory_path)

# Verify the change
current_directory = os.getcwd()
print("Current working directory:", current_directory)


from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal
from rdflib.plugins import stores
import csv



Current working directory: C:\backup\narrateastory


In [2]:
import csv
from rdflib import Graph, Literal, Namespace, RDF, RDFS, OWL, URIRef

# Create the RDF graph and define the namespaces
rdf_graph = Graph()
ex = Namespace("http://narrateastory.com/heritageontology#")

rdf_graph.bind("ex", ex)
rdf_graph.bind("owl", OWL)
rdf_graph.bind("rdfs", RDFS)

# Define inferred_triples at a higher scope
inferred_triples = set()

# Function to infer transitive closure
def infer_transitive_closure(graph, property_uri, visited_triples=None):
    if visited_triples is None:
        visited_triples = set()

    global inferred_triples  # Reference the global variable
    for s1, p, o1 in graph.triples((None, property_uri, None)):
        for s2, _, o2 in graph.triples((None, property_uri, s1)):
            inferred_triple = (s2, property_uri, o1)
            if inferred_triple not in visited_triples:
                graph.add(inferred_triple)
                inferred_triples.add(inferred_triple)
                print(f"Inferred: {inferred_triple[0]} {inferred_triple[1]} {inferred_triple[2]}")
                visited_triples.add(inferred_triple)
                # Recursively check for more transitive closure
                infer_transitive_closure(graph, property_uri, visited_triples)

# Read the CSV file and build the ontology
with open("markers_data.csv", "r", encoding="utf-8-sig") as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        subject = row["subject"].replace(" ", "_")
        relation = row["relation"]
        obj = row["obj"].replace(" ", "_")

        subject_uri = ex[subject]
        obj_uri = ex[obj]

        # Check if the relation is "isa" to determine superclass-subclass relationship
        if relation.lower() == "isa":
            # Check if the subclass relation already exists to avoid conflicts
            if (subject_uri, RDFS.subClassOf, obj_uri) not in rdf_graph and (obj_uri, RDFS.subClassOf, subject_uri) not in rdf_graph:
                rdf_graph.add((subject_uri, RDF.type, OWL.Class))
                rdf_graph.add((obj_uri, RDF.type, OWL.Class))
                rdf_graph.add((obj_uri, RDFS.subClassOf, subject_uri))
                inferred_triples.add((subject_uri, RDFS.subClassOf, obj_uri))
            else:
                print(f"Conflict detected: {subject} {relation} {obj}. Skipping.")

        else:
            # Check if the reverse relation exists to avoid conflicts
            if (obj_uri, ex[relation], subject_uri) not in rdf_graph:
                # Add subject and obj classes
                rdf_graph.add((subject_uri, RDF.type, OWL.Class))
                rdf_graph.add((obj_uri, RDF.type, OWL.Class))

                # Add relation property
                relation_uri = ex[relation]
                rdf_graph.add((relation_uri, RDF.type, OWL.ObjectProperty))

                # Add individual instances
                rdf_graph.add((subject_uri, RDF.type, subject_uri))
                rdf_graph.add((obj_uri, RDF.type, obj_uri))

                # Add relation between subject and obj classes
                rdf_graph.add((subject_uri, relation_uri, obj_uri))

                # Add relation between subject and obj instances
                rdf_graph.add((subject_uri, relation_uri, obj_uri))
                inferred_triples.add((subject_uri, relation_uri, obj_uri))

                # Add additional properties as RDFS:seeAlso
                if "timePeriod" in row:
                    rdf_graph.add((subject_uri, RDFS.seeAlso, Literal(row["timePeriod"])))
                if "lat" in row and "long" in row:
                    rdf_graph.add((subject_uri, RDFS.seeAlso, Literal(f"Lat: {row['lat']}, Long: {row['long']}")))
                if "utube_link" in row:
                    rdf_graph.add((subject_uri, RDFS.seeAlso, URIRef(row["utube_link"])))
                if "further_reading" in row:
                    rdf_graph.add((subject_uri, RDFS.seeAlso, URIRef(row["further_reading"])))
                if "current_name_of_place" in row:
                    rdf_graph.add((subject_uri, RDFS.seeAlso, Literal(row["current_name_of_place"])))

            else:
                print(f"Conflict detected: {subject} {relation} {obj}. Skipping.")

# Infer transitive closure for "hasAncientTradeRoute" (not "hasAncientTradeRoutes")
infer_transitive_closure(rdf_graph, ex.hasAncientTradeRoute)

# Save the entire inferred graph to an RDF file
rdf_graph.serialize(destination="inferredtriples.owl", format="xml")

# Save the inferred triples to a CSV file
with open("inferredtriples.csv", "a", encoding="utf-8-sig", newline="") as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=",")
    for inferred_triple in inferred_triples:
        if not inferred_triple[0].endswith("_instance") and not inferred_triple[2].endswith("_instance"):
            csv_writer.writerow([inferred_triple[0].split("#")[1], inferred_triple[1].split("#")[1], inferred_triple[2].split("#")[1]])


Inferred: http://narrateastory.com/heritageontology#nathula http://narrateastory.com/heritageontology#hasAncientTradeRoute http://narrateastory.com/heritageontology#Tamralipta
Inferred: http://narrateastory.com/heritageontology#nathula http://narrateastory.com/heritageontology#hasAncientTradeRoute http://narrateastory.com/heritageontology#Saptagram
